# Model Evaluation for Intrusion Detection System

This notebook provides detailed evaluation metrics, confusion matrices, and performance analysis for trained IDS models.


In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

from src.preprocessing import DataPreprocessor
from src.models import RandomForestIDS, SVMIDS, NeuralNetworkIDS
from src.evaluation import ModelEvaluator
from src.visualization import IDSVisualizer

print("Libraries imported successfully!")


## 1. Load Models and Test Data


In [ ]:
# Load preprocessor
preprocessor = DataPreprocessor()
preprocessor.load_preprocessor('../models/preprocessor.pkl')

# Load test data
X_test = pd.read_csv('../data/processed/X_test.csv')
y_test = pd.read_csv('../data/processed/y_test.csv').values.ravel()

print(f"Test set shape: {X_test.shape}")
print(f"Number of test samples: {len(y_test)}")

# Get class names
class_names = preprocessor.label_encoder.classes_ if hasattr(preprocessor.label_encoder, 'classes_') else None
print(f"Classes: {class_names}")


## 2. Evaluate Random Forest Model


In [ ]:
# Load model
rf_model = RandomForestIDS()
rf_model.load('../models/random_forest_model.pkl')

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_proba_rf = rf_model.predict_proba(X_test)

# Evaluate
rf_evaluator = ModelEvaluator("Random Forest")
rf_metrics = rf_evaluator.evaluate(y_test, y_pred_rf, y_proba_rf)
rf_evaluator.confusion_matrix(y_test, y_pred_rf, class_names=class_names)

# ROC curve (for binary classification)
if len(np.unique(y_test)) == 2:
    rf_evaluator.roc_curve(y_test, y_proba_rf)


## 3. Evaluate SVM Model


In [ ]:
# Load model
svm_model = SVMIDS()
svm_model.load('../models/svm_model.pkl')

# Make predictions
y_pred_svm = svm_model.predict(X_test)
y_proba_svm = svm_model.predict_proba(X_test)

# Evaluate
svm_evaluator = ModelEvaluator("SVM")
svm_metrics = svm_evaluator.evaluate(y_test, y_pred_svm, y_proba_svm)
svm_evaluator.confusion_matrix(y_test, y_pred_svm, class_names=class_names)

# ROC curve (for binary classification)
if len(np.unique(y_test)) == 2:
    svm_evaluator.roc_curve(y_test, y_proba_svm)


## 4. Evaluate Neural Network Model


In [ ]:
# Load model
nn_model = NeuralNetworkIDS()
nn_model.load('../models/neural_network_model.h5')

# Make predictions
y_pred_nn = nn_model.predict(X_test)
y_proba_nn = nn_model.predict_proba(X_test)

# Evaluate
nn_evaluator = ModelEvaluator("Neural Network")
nn_metrics = nn_evaluator.evaluate(y_test, y_pred_nn, y_proba_nn)
nn_evaluator.confusion_matrix(y_test, y_pred_nn, class_names=class_names)

# ROC curve (for binary classification)
if len(np.unique(y_test)) == 2:
    nn_evaluator.roc_curve(y_test, y_proba_nn)


## 5. Comprehensive Model Comparison


In [ ]:
# Collect all metrics
all_metrics = {
    'Random Forest': rf_metrics,
    'SVM': svm_metrics,
    'Neural Network': nn_metrics
}

# Create comparison
comparison_df = pd.DataFrame(all_metrics).T
print("Comprehensive Model Comparison:")
print(comparison_df)

# Visualize
visualizer = IDSVisualizer()
visualizer.plot_model_performance_comparison(all_metrics)

# Save results
comparison_df.to_csv('../models/detailed_evaluation.csv')
print("\nDetailed evaluation saved to ../models/detailed_evaluation.csv")
